<a href="https://colab.research.google.com/github/JohnZ03/Open-L2O/blob/SpecialCaseK%3D1/Gaussian_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# Set environments
%tensorflow_version 2.6

# Check python versiom
!python3 --version

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.6`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
Python 3.7.11


In [35]:
# Import packages
import os
import tensorflow as tf
import cProfile

import matplotlib.pyplot as plt
import numpy as np

from scipy.linalg import null_space # For calculating null space of H

from scipy.optimize import fsolve, root, minimize # non-linear funtion solver

# Enable Eager Execution
tf.executing_eagerly()

# Uncomment to see where your variables get placed
# tf.debugging.set_log_device_placement(True)

True

In [36]:
# def quadFunctionGroup(alpha, A):
#   n = A.shape[2]
#   y = np.zeros(n+1)
#   for i in range(n):
#     y[i] = np.matmul(np.matmul(alpha, A[:,:,i]), alpha.transpose()) - 1
#     y[n] += y[i]
#   return y

def quadFunctionGroup(alpha, A):
  n = A.shape[2]
  y = 0
  for i in range(n):
    y += np.matmul(np.matmul(alpha, A[:,:,i]), alpha.transpose())
  return y

In [37]:
def main():
  # Generate H with normal distribution
  user_count = 20
  antenna_count = 50

  H_real = np.random.normal(size=(user_count, antenna_count))
  H_imag = np.random.normal(size=(user_count, antenna_count))
  H = tf.complex(H_real, H_imag)

  # H = [[1.419310150642549 - 1.147952778898594j, 0.197811053464361 + 0.722254032225002j, -0.804465956349547 - 0.666890670701386j, 0.835088165072682 - 0.082494425370955j, 0.215670086403744 - 0.438966153934773j], \
  #      [0.291584373984183 + 0.104874716016494j, 1.587699089974059 + 2.585491252616242j, 0.696624415849607 + 0.187331024578940j, -0.243715140377952 - 1.933022917850987j, -1.165843931482049 - 1.794678841455123j]]

  H = np.array(H)

  dimNull = antenna_count - user_count
  H_null = null_space(H)
  H_null_real = tf.math.real(H_null)
  H_null_imag = tf.math.imag(H_null)

  H_combi1 = np.zeros((antenna_count, 2*dimNull))
  H_combi2 = np.zeros((antenna_count, 2*dimNull))
  for i in range(dimNull):
    H_combi1[:, 2*i] = H_null_real[:, i]
    H_combi1[:, 2*i+1] = -1 * H_null_imag[:, i]

    H_combi2[:, 2*i] = H_null_imag[:, i]
    H_combi2[:, 2*i+1] = H_null_real[:, i]

  A = np.zeros((2*dimNull, 2*dimNull, antenna_count))
  for i in range(antenna_count):
    A[:,:,i] = np.matmul(H_combi1[i,:].transpose(),H_combi1[i,:]) \
          + np.matmul(H_combi2[i,:].transpose(),H_combi2[i,:])

  alpha0 = np.ones((1, 2*dimNull))

  OptimizeResult = minimize(quadFunctionGroup, alpha0, args=(A))

  alphaResult = OptimizeResult.x

  # Calculate for x
  alphaRe = np.zeros(dimNull)
  alphaIm = np.zeros(dimNull)

  for i in range(dimNull):
    alphaRe[i] = alphaResult[i*2]
    alphaIm[i] = alphaResult[i*2+1]

  alphaComplex = tf.complex(alphaRe, alphaIm)

  x = H_null @ np.conjugate(alphaComplex).transpose()

  return (tf.math.real(tf.norm(tf.reduce_sum(H@x))).numpy(), x)

In [38]:
%%time
sum = 0
x_list = []
for n in range(100):
  sum += main()[0]
  x_list.append(main()[1])
print(sum)

1.1139739494811895e-14
CPU times: user 53 s, sys: 175 ms, total: 53.2 s
Wall time: 53.1 s


In [39]:
# print(alphaResult)
# print(x)
# print(H)
# print(OptimizeResult.success)
# print(OptimizeResult.fun)
# print(tf.math.real(tf.norm(tf.reduce_sum(H@x))).numpy())

In [40]:
print(x_list)

[array([-0.01103012+1.02411685e-03j,  0.00792257+1.08816285e-02j,
       -0.00125371-5.37483644e-03j, -0.00320131-1.19715183e-02j,
        0.01931366-4.11129618e-04j, -0.00526433-6.04459443e-04j,
        0.0053233 -6.04846454e-03j,  0.00707318-1.61400220e-03j,
        0.00128292+2.57233495e-03j, -0.00301328+6.96720056e-04j,
        0.00164724-3.22155710e-03j,  0.01558984-7.61632024e-03j,
       -0.01091067+7.19597837e-03j,  0.00737167+8.16744823e-03j,
        0.00116158+1.37945483e-03j, -0.00799602+5.17754034e-03j,
       -0.00093688-2.19818642e-03j,  0.00045423+9.55974588e-04j,
        0.00349908-1.05432204e-03j, -0.00277578+1.01434434e-02j,
       -0.00303894+1.68605515e-03j, -0.0074088 +6.25074635e-03j,
       -0.00066224+1.28123464e-02j, -0.00715202+5.55874441e-03j,
       -0.006741  +8.29790125e-03j, -0.00203342+6.41341122e-03j,
       -0.00546555-3.42042340e-03j, -0.00333393-4.29840067e-03j,
        0.00710748-5.35997230e-03j,  0.00452512+2.84142296e-04j,
        0.00149462-1.068